Fair warning: not my most beautiful code!
-Dan 

# Imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from palettable.cmocean.diverging import Delta_10 as rgb
from palettable.cmocean.diverging import Curl_20 as rgb
# from palettable.cmocean.sequential import Gray_20 as rgb
cm = rgb.mpl_colormap
import matplotlib as mpl
from scipy.stats import norm as normal

# Get the data

In [16]:
switch = pd.read_csv('../../data/output/switch_counts_2019_12_12.csv')
survey = pd.read_csv('../../data/processed/survey.csv')
# Make names of fields lowercase
survey['Discipline'] = survey['Discipline'].apply(lambda x: x.lower())
counts = pd.read_csv('../../data/output/field_counts_2019_12_12.csv')
counts['ratio'] = counts['n_female']/(counts['n_male']+counts['n_female'])
fields = switch['to_matched_field'].unique()
fields.sort()
survey['Long Discipline'] = fields
fabs = survey['FAB']

In [18]:
def write_circle(file,center,radius,stroke,fillcolor,strokecolor):
    file.write('<circle class="node" r="{}" style="fill: rgb({},{},{}); stroke: rgb({},{},{}); stroke-width:{}" cx="{}" cy="{}"/>\n'
               .format(radius,fillcolor[0],fillcolor[1],fillcolor[2],
                       strokecolor[0],strokecolor[1],strokecolor[2],
                       stroke,center[0],center[1]))
def write_arc(file,centerFrom,centerTo,anchor,width,color,opacity):
    file.write('<path d="M{} {} Q {} {} {} {}" stroke-width="{}" stroke="rgb({},{},{})" stroke-opacity="{}" fill="none"/>\n'
               .format(
                   centerFrom[0],centerFrom[1],
                   anchor[0],anchor[1],
                   centerTo[0],centerTo[1],
                   width,
                   color[0],color[1],color[2],
                   opacity))
def write_text(file,xy,myText,myClass,myRotate,myTranslate):
    file.write('<text x="{}" y="{}" class="{}" transform="translate({},{}) rotate({} {},{})">{}</text>\n'.format(
        xy[0],xy[1],
        myClass,
        myTranslate[0],myTranslate[1],
        myRotate,xy[0],xy[1],
        myText))
def write_line(file,myStart,myFinish,stroke,color,opacity):      
    file.write('<line x1="{}" y1="{}" x2="{}" y2="{}" stroke-width="{}" stroke="rgb({},{},{})" stroke-opacity="{}"/>\n'.format(
        myStart[0],myStart[1],
        myFinish[0],myFinish[1],
        stroke,color[0],color[1],color[2],opacity))

In [19]:
# Custom colormaps
# ACCENT_COLOR_1 = np.array([249, 79, 21])/255
ACCENT_COLOR_1 = np.array([0, 0, 255])/255
ACCENT_COLOR_2 = np.array([255, 0, 0])/255
r,g,b = ACCENT_COLOR_1
R,G,B = ACCENT_COLOR_2
t = 0.750
cdict = {'red':   ((0.0,  r, r),
                   (0.5,  t, t),
                   (1.0,  R, R)),
         'green': ((0.0,  g, g),
                   (0.5,  t, t),
                   (1.0,  G, G)),
         'blue':  ((0.0,  b, b),
                   (0.5,  t, t),
                   (1.0,  B, B))}
CMAP_AC1_TO_ALMOSTBLACK_TO_AC2 = mpl.colors.LinearSegmentedColormap('testcmap', cdict)
color_range = [-0.4,0.4]
norm = mpl.colors.Normalize(vmin=color_range[0], vmax=color_range[1])

In [20]:
def get_edges(fields,df,counts):
    edges = []
    for fr in fields:
        for to in fields:
            n_men = df.loc[(df['from_matched_field']==fr) 
                               & (df['to_matched_field']==to),'n_male'].values
            n_women = df.loc[(df['from_matched_field']==fr) 
                                 & (df['to_matched_field']==to),'n_female'].values
            n_men_origin = counts.loc[counts['field']==fr,'n_male'].values
            n_women_origin = counts.loc[counts['field']==fr,'n_female'].values
            if len(n_men)==0:
                n_men = 0
            if len(n_women)==0:
                n_women = 0
            if len(n_men_origin)==0:
                n_men_origin = 0
            if len(n_women_origin)==0:
                n_women_origin = 0
            if (n_men+n_women)==0:
                continue
            else:
                ratio_flow = n_women/(n_men+n_women)
                edge = {}
                edge['men'] = n_men[0]
                edge['women'] = n_women[0]
                edge['ratio'] = ratio_flow[0]
                edge['men_origin'] = n_men_origin[0]
                edge['women_origin'] = n_women_origin[0]
                edge['ratio_origin'] = n_women_origin[0]/(n_men_origin[0]+n_women_origin[0])
                edge['ratio-diff'] = edge['ratio']-edge['ratio_origin']
                edge['fr'] = fr
                edge['to'] = to
                edge['color'] = 255*get_color(edge['ratio-diff'])
                edges.append(edge.copy())
    return edges

In [21]:
def get_color(t):
    return np.array(CMAP_AC1_TO_ALMOSTBLACK_TO_AC2(np.interp(t,color_range,[0,1])))

In [22]:
def svg_circle(center,radius,stroke,fillcolor,strokecolor):
    circle =  '<circle class="node" r="{}" style="fill: rgb({},{},{}); stroke: rgb({},{},{}); stroke-width:{}" cx="{}" cy="{}"/>\n'.format(
        radius,fillcolor[0],fillcolor[1],fillcolor[2],
        strokecolor[0],strokecolor[1],strokecolor[2],
        stroke,center[0],center[1])
    return circle
def svg_line(myStart,myFinish,stroke,color,opacity):      
    line = '<line x1="{}" y1="{}" x2="{}" y2="{}" stroke-width="{}" stroke="rgb({},{},{})" stroke-opacity="{}"/>\n'.format(
        myStart[0],myStart[1],
        myFinish[0],myFinish[1],
        stroke,color[0],color[1],color[2],opacity)
    return line
def svg_text(xy,myText,myClass,myRotate,myTranslate):
    text = '<text x="{}" y="{}" class="{}" transform="translate({},{}) rotate({} {},{})">{}</text>\n'.format(
        xy[0],xy[1],
        myClass,
        myTranslate[0],myTranslate[1],
        myRotate,xy[0],xy[1],
        myText)
    return text
def svg_arc(centerFrom,centerTo,anchor,width,color,opacity):
    arc = '<path d="M{} {} Q {} {} {} {}" stroke-width="{}" stroke="rgb({},{},{})" stroke-opacity="{}" fill="none"/>\n'.format(
        centerTo[0],centerTo[1],
        anchor[0],anchor[1],
        centerFrom[0],centerFrom[1],
        width,
        color[0],color[1],color[2],
        opacity)
    return arc

In [23]:
class RankSVG:
    def __init__(self, path, width=600, aspectRatio=9/5, swirl=1/4, pad=230, text_offset=180, text_size=12):
        self.path = path
        self.width = int(width)
        self.height = int(width*aspectRatio)
        self.aspectRatio = aspectRatio
        self.swirl = swirl
        self.pad = pad
        self.text_offset = text_offset
        self.text_size = text_size
        
        self.nodes = []
        self.node2center = {}
        self.node2size = {}
        
        self.edges = []
        self.set_head()    
    
    def get_anchor(self,fr,to):
        anchor_y = (1-self.swirl)*fr + self.swirl*to;
        anchor_x = (self.width/2)+np.sign(to-fr)*np.abs(to-fr)/self.aspectRatio;
        return [anchor_x,anchor_y]
    
    def set_head(self):
        self.head = '<svg width="{}" height="{}" id="{}" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink">\n'.format(self.width,self.height,'v1') + '''
        <style>
            .sans { font: 12px sans-serif; }
            .grey { stroke: rgb(150,150,150); stroke-width:0.5;}
        </style>
        '''
    
    def set_nodes(self,nodes):
        self.nodes = [node for node in nodes]
        
    def add_edge(self,edge):
        self.edges.append(edge.copy())
        
    def get_nodes(self):
        return self.nodes
    
    def set_node_centers(self,node2center):
        for key in node2center.keys():
            self.node2center[key] = [self.width/2,
                                     np.interp(node2center[key],
                                               (np.min(list(node2center.values())),
                                                np.max(list(node2center.values()))),
                                               (self.height-self.pad,self.pad))]
    
    def set_node_sizes(self,node2size):
        for key in node2size.keys():
            self.node2size[key] = node2size[key]
            
    def set_axis_label(self,axis_label):
        self.axis_label = axis_label
            
    def write_nodes(self,f):
        for node in self.nodes:
            circle = svg_circle(
                center=self.node2center[node],
                radius=self.node2size[node],
                stroke=1.5,
                fillcolor=[100,100,100],
                strokecolor=[255,255,255])
            f.write(circle)
            
    def write_center_line(self,f):
        center_line = svg_line(
            [self.width/2,
             np.min(list(self.node2center.values()))-self.pad/4],
            [self.width/2,
             np.max(list(self.node2center.values()))+self.pad/4],
            stroke=0.5,
            color=np.array([150,150,150]),
            opacity=1)
        f.write(center_line)
    
    def label_nodes(self,f):
        self.write_label_lines(f)
        self.write_label_text(f)
    
    def write_label_lines(self,f):
        for node in self.nodes:
            label_line = svg_line(
                self.node2center[node],
                np.array(self.node2center[node])+np.array([self.text_offset,0]),
                stroke=0.5,
                color=np.array([150,150,150]),
                opacity=1)
            f.write(label_line)
    
    def write_label_text(self,f):
        for node in self.nodes:
            text = svg_text(self.node2center[node],
                            myText=node,
                            myTranslate=[self.text_offset,self.text_size/4],
                            myRotate=0,myClass="sans")
            f.write(text)
            
    def label_axis(self,f):
        axis_text = svg_text(
            [self.width/2-4,self.pad-10],self.axis_label,
            'sans',-90,[0,0])
        f.write(axis_text)
    
    def write_under_edges(self,f):
        rgb = np.array([0,0,0])
        for edge in self.edges:
            if 'width' in edge.keys():
                w = edge['width']
            else:
                w = 2
            under_arc = svg_arc(self.node2center[edge['fr']],
                          self.node2center[edge['to']],
                          self.get_anchor(self.node2center[edge['fr']][1],
                                          self.node2center[edge['to']][1]),
                          width=w+2,
                          color=np.array([255,255,255]),
                          opacity=1)
            f.write(under_arc)
    
    def write_edges(self,f):
        rgb = np.array([0,0,0])
        for edge in self.edges:
            if 'width' in edge.keys():
                w = edge['width']
            else:
                w = 2
            if 'color' in edge.keys():
                c = edge['color']
            else:
                c = np.array([100,100,100])
            if 'alpha' in edge.keys():
                alph = edge['alpha']
            else:
                alph = 0.3
            arc = svg_arc(self.node2center[edge['fr']],
                          self.node2center[edge['to']],
                          self.get_anchor(self.node2center[edge['fr']][1],
                                          self.node2center[edge['to']][1]),
                          width=w,
                          color=c,
                          opacity=alph)
            f.write(arc)
    
    def dump(self):
        f = open(self.path,'w')
        f.write(self.head)
        self.write_center_line(f)
        self.label_nodes(f)
        self.label_axis(f)
        self.write_under_edges(f)
        self.write_edges(f)
        self.write_nodes(f)
        f.write('</svg>')
        f.close()

# Create Viz

In [24]:
x_strings = ['FAB','SEL-A','S vs E','HW-OC']
for survey_variable in x_strings:
    svg = RankSVG('network_viz/{}_v4.2.svg'.format(survey_variable))
    svg.set_nodes(fields)
    svg.set_axis_label(survey_variable)

    node2center = {}
    for field in fields:
        node2center[field] = survey.loc[survey['Long Discipline']==field,survey_variable].values[0]
    svg.set_node_centers(node2center)

    node2size = {}
    for field in fields:
        node2size[field] = 3.5
    svg.set_node_sizes(node2size)

    edges = get_edges(fields,switch,counts)

    for edge in edges:
        se = float(
            np.sqrt(
                edge['ratio']*(1-edge['ratio'])
                /(edge['men']+edge['women'])
                +edge['ratio_origin']*(1-edge['ratio_origin'])
                /(edge['men_origin']+edge['women_origin'])))
        if np.abs(edge['ratio-diff']) - normal.ppf(0.975)*se > 0:
            edge['width'] = 2.5
            edge['alpha'] = 0.4
            svg.add_edge(edge)
        else:
            edge['width'] = 1
            edge['alpha'] = 0.3
#             svg.add_edge(edge)
# do nothing

    svg.dump()
